In [1]:
# Import packages
import pandas as pd
import os

# Extract

In [2]:
customer_support_path = os.path.join(
    os.path.abspath('../'),
    'data',
    'raw',
    'customer_suport_on_twitter',
    'twcs'
)

In [3]:
df_customer_support = pd.read_csv(
    os.path.join(customer_support_path, 'twcs.csv')
)

df_customer_support.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


# Transform
Considering that each thread starts with an tweet without a in_response_to_tweet_id (NaN), the thread_id should start in this row and the previous rows will be the same thread

## Filter dataset

In [4]:
# Convert created_at column to datetime
df_customer_support['created_at'] = pd.to_datetime(
    df_customer_support['created_at'], format='%a %b %d %H:%M:%S %z %Y'
)

# Filter period
df_customer_support = df_customer_support[
    df_customer_support['created_at'].between('2008-06-08', '2016-07-01')
]

## Create columns

In [5]:
# Create thread_start column to identify if it is the first tweet of a thread or not
df_customer_support.loc[
    df_customer_support['in_response_to_tweet_id'].isna(),
    'thread_start'
] = True

# Set the date the thread start date
df_customer_support.loc[
    df_customer_support['in_response_to_tweet_id'].isna(),
    'thread_start_date'
] = df_customer_support['created_at'].dt.date

# Set the thread_id
df_customer_support['thread_id'] = df_customer_support['thread_start'].cumsum()
df_customer_support['thread_id'] = df_customer_support['thread_id'].fillna(method='backfill')

/tmp/ipykernel_422366/2651857295.py:15: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_customer_support['thread_id'] = df_customer_support['thread_id'].fillna(method='backfill')


## Indetify companies

In [6]:
# Identify the company involved in each thread
df_companies = df_customer_support.loc[
    ~df_customer_support['author_id'].str.contains(r'[0-9]{4,6}'),
    ['author_id', 'thread_id']
].drop_duplicates('thread_id')

# Rename column
df_companies.rename({'author_id': 'company'}, axis=1, inplace=True)

# Merge with df_customer_support
df_customer_support = df_customer_support.merge(df_companies, on='thread_id', how='left')

## Compute thread metrics and join tweets text
<strong>Metrics</strong> -> Number of tweets | max date | min date

In [7]:
# Define parameters for aggregate function
agg_params = {
    'tweet_id': 'count',
    'created_at': ['max', 'min'],
    'text': ' '.join
}

# Group by thread to count tweets and calculate the thread's duration
df_threads = (
    df_customer_support
    .groupby('thread_id', as_index=False)
    .agg(agg_params)
)

# Rename columns
df_threads.columns = ['thread_id', 'n_tweets', 'ended_at', 'started_at', 'thread_text']

# Calculate the duration of each thread
df_threads['duration'] = (
    df_threads['ended_at'] - df_threads['started_at']
).dt.total_seconds().div(60).astype(int)

# Organize columns
df_threads = df_threads[
    [
        'thread_id',
        'n_tweets',
        'ended_at',
        'started_at',
        'duration',
        'thread_text'
    ]
]

# Merge with df_customer_support
df_customer_support = df_customer_support.merge(df_threads, on='thread_id', how='left')

In [8]:
df_threads

,thread_id,n_tweets,ended_at,started_at,duration,thread_text
0,True,1,2011-08-29 03:20:05+00:00,2011-08-29 03:20:05+00:00,0,"Lost your booking number? No worries, just dro..."
1,2.0,4,2014-12-04 17:57:42+00:00,2014-12-04 00:22:00+00:00,1055,@119616 The .nbi files are part of the backup ...
2,3.0,1,2013-12-13 15:31:53+00:00,2013-12-13 15:31:53+00:00,0,Unreservable coaches with the new timetable ar...
3,4.0,2,2014-08-27 07:29:33+00:00,2014-08-27 06:31:06+00:00,58,"Hello @121643 , please provide the affected mo..."
4,5.0,2,2016-02-07 10:47:37+00:00,2016-02-07 10:40:43+00:00,6,@121880 The Premium subscription 21660 has bee...
...,...,...,...,...,...,...
423,424.0,6,2015-12-11 20:44:39+00:00,2015-12-11 20:38:22+00:00,6,@817936 If you put reason as same day refund a...
424,425.0,2,2016-04-15 10:15:53+00:00,2015-09-06 18:05:56+00:00,319209,"@818934 Hello Mary, we just reply to your DM. ..."
425,426.0,3,2014-12-09 17:05:05+00:00,2014-12-09 11:54:41+00:00,310,@822638 Frozen on message: Preparing to turn o...
426,427.0,2,2015-11-20 13:07:03+00:00,2015-11-20 08:15:51+00:00,291,"@822842 We'd like to look into this, Gordan. C..."


## Compute company metrics per date

In [9]:
# Sort values to join texts in the correct order
df_customer_support.sort_values(['thread_id', 'created_at'])

# Define parameters for aggregate function
agg_func = {
    'thread_id': 'count',
    'n_tweets': ['sum', 'mean'],
    'duration': 'mean'
}

# Group by company and thread start date to compute
# number of threads, tweets, duration and join tweets text
df_customer_support_result = (
    df_customer_support
    .groupby(['company', 'thread_start_date'], as_index=False)
    .agg(agg_func)
)

# Rename columns
df_customer_support_result.columns = [
    'company',
    'thread_start_date',
    'n_threads',
    'n_tweets',
    'avg_tweets_per_thread',
    'avg_duration_per_thread'
]

# Load

In [10]:
df_threads.to_csv(
    os.path.join(
        os.path.abspath('../'),
        'data',
        'processed',
        'customer_support_on_twitter_threads_text.csv'
    )
)

In [11]:
df_customer_support_result.to_csv(
    os.path.join(
        os.path.abspath('../'),
        'data',
        'processed',
        'customer_support_on_twitter.csv'
    )
)